# b1_extract_all_paragraphes
From the list_meetings.csv extract all the paragraphes in the html page for each issues and create a file paragraphes.txt with it 

In [1]:
import urllib
from urllib.request import urlopen
import requests
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
from urllib.request import urlopen, Request
import csv
import requests

In [2]:
def write_paragraphe(paragraphes):
    """ Write "paragraphes" into a file paragraphes.txt. """
    paragraphes = list(set(paragraphes))
    # generate list_p_tags.txt file
    outF = open("paragraphes"+str(number)+".txt", "w")
    for line in paragraphes:
        outF.write(str(line))
        outF.write("\n")
    outF.close()

In [3]:
def remove_unwanted_p(paragraphes):
    """ Remove all the text into paragraphes that is no needed."""
    paragraphes = [p.replace('<p align="justify">','') for p in paragraphes]
    paragraphes = [p.replace('<p align="CENTER">','') for p in paragraphes] 
    #paragraphes = [re.sub('<strong>.+</strong>',' ',str(p)) for p in paragraphes]
    paragraphes = [re.sub('<p align="justify" class="ENB-Body">','',p) for p in paragraphes]
    paragraphes = [re.sub('<font face="Verdana" size="2">','',p) for p in paragraphes]
    paragraphes = [re.sub(r'<font.*?>','',p) for p in paragraphes]
    paragraphes = [re.sub(r'<a.*?>','',p) for p in paragraphes]
    paragraphes = [re.sub(r'<.*?>','',p) for p in paragraphes]
    #paragraphes = [p for p in paragraphes if not p.isupper()] 
    return paragraphes

In [4]:
def find_bad_html(page_string):
    paragraphes = re.split(r'<p>|</p>|<b>|</b>|<p ',page_string)
    start = "<p>"
    end = "</p>"
    page_string = [start+ p+ end for p in paragraphes]
    return page_string

In [5]:
def clean_page_to_parse(page_string):
    page_string = find_bad_html(str(page_string))
    """ Remove comment at the beginning of the hml and also put the <html> tag in lowercase and <b> tags to be able to collect all paragraphes. """
    page_string = re.sub('<!-- WWW Designer Jeff Anderson janderson@iisd.ca --!>','',str(page_string))
    page_string = re.sub('<!-- WWW Designer Jeff Anderson janderson@iisd.ca --!>','',str(page_string))
    page_string = re.sub('<!-- WWW design Jeff Anderson janderson@iisd.ca ---!>','',str(page_string))
    page_string = re.sub("<!--.*?<html", '<html', page_string, flags=re.MULTILINE)

    page_string = re.sub(r'<HTML>',r'<html>',str(page_string))
    page_string = re.sub(r'</HTML>',r'</html>',str(page_string))

    return bytes(page_string,'utf-8')

In [6]:
def remove_special_char(p):
    """ Remove special char to be able to detect easier sentences. """
    s = p.replace("\r"," ")
    s = s.replace(r'\x'," ")
    s = s.replace("\n"," ")
    s = s.replace("\t"," ")
    s = s.replace("\\x"," ")
    s = s.replace("\\r"," ")
    s = s.replace("\\n"," ")
    s = s.replace("\\t"," ")
    s = s.replace("\\"," ")
    q = re.compile('\s\s+')
    p = re.compile('\n\n+')
    s = q.sub(' ',s)
    s = p.sub(' ',s)
    #s = re.split('<p>',s)
    return s

In [7]:
def remove_footer(paragraphes):
    """ Remove footer of the page with conditions. """
    k = len(paragraphes)
    for i in range(len(paragraphes)):
        if('IN THE CORRIDORS' in paragraphes[i] or 'THINGS TO LOOK'  in paragraphes[i] or 'This issue of' in paragraphes[i]  or 'BRIEF ANALYSIS OF' in paragraphes[i]):
            k=i
            break

    return paragraphes[:k]

In [8]:
def extract_p_tags(html_link):
    """ Extract the <p> tag from a specific html link. """
    #Parse the page 
    r = Request(html_link, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(r).read()
    page = clean_page_to_parse(page)
    soup = soup = BeautifulSoup(page, 'html.parser')

    list_tp = soup.find_all('p',recursive=False)
    if(len(list_tp) == 0):

        x = "."
        list_tp = list(soup.find_all('p'))
    #Extract all the text and remove undesired paragraphes
    list_tp2 = []
    for p in list_tp:
        list_tp2 += re.split('<p>',remove_special_char(str(p)))
    list_tp = remove_footer(list_tp2)
    # Remove titles and sentences all in uppercase
    list_tp = remove_unwanted_p(list_tp)
    return list_tp

In [9]:
def extract_p_tags_411(html_link):
    """ Extract the <p> tag from a specific html link. """
    #Parse the page 
    r = Request(html_link, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(r).read()
    page = clean_page_to_parse(page)
    soup = soup = BeautifulSoup(page, 'html.parser')

    list_tp = soup.find_all('td',recursive=False)
    if(len(list_tp) == 0):

        x = "."
        list_tp = list(soup.find_all('td'))
    #Extract all the text and remove undesired paragraphes
    list_tp2 = []
    
    for p in list_tp:
        list_tp2 += re.split(r'<br/><br/>|<br /><br />',remove_special_char(str(p)))
    list_tp = remove_footer(list_tp2)
    # Remove titles and sentences all in uppercase
    list_tp = remove_unwanted_p(list_tp)
    return list_tp

In [10]:
def extract_p_tags_45(html_link):
    """ Extract <p> tags from link inside "html_link" for Issue# <45. """
    r = Request(html_link, headers={'User-Agent': 'Mozilla/5.0'})
    page_link = urlopen(r).read()
    #page_link = urllib.request.urlopen(html_link).read()
    soup_link = BeautifulSoup(page_link)
    paragraphes = soup_link.findAll('a',href = re.compile('\d+'))
    list_tp = []

    for pa in paragraphes:
        # doesn't use the link of the main page (all issues)
        if(pa['href'] != '1200000e.html'):
            html_link = 'https://enb.iisd.org/vol12/'+pa['href']
            list_tp += extract_p_tags(html_link)

    return list_tp

In [11]:
def extract_from_csv_list_issues(csv_file):
    """ Extract from "csv_file" all the html link to be able to extract all the <p> tags. """
    f = open(csv_file)
    csv_f = csv.reader(f)
    list_pt = []
    return list(csv_f)[1:]

In [12]:
def write_list_pargraphes(list_paragraphes):
    """ Generate list_p_tags.txt file. """
    outF = open("paragraphes.txt", "w")
    for line in list_paragraphes:
        outF.write(str(line))
        outF.write("\n")
    outF.close()

In [13]:
def extract_from_csv_p_tags(csv_file):
    """ Extract from "csv_file" all the html link to be able to extract all the <p> tags. """
    list_meetings = extract_from_csv_list_issues(csv_file)
    list_pt = []
    for i, row in enumerate(list_meetings):

        #Extract for 0 < Issue# < 45 
        if(int(row[4]) < 45 ):
            list_pt += extract_p_tags_45(row[6])

        #Extract for 66 < Issue# < 775
        else :
            list_pt = []
            url =  row[6]
            
            request = requests.get(url)
            if(int(row[4]) != 175 and int(row[4]) != 300 and request.status_code == 200):
                if(411 <= int(row[4]) and int(row[4]) <= 420):
                    list_pt += extract_p_tags_411(row[6])
                else:
                    list_pt = extract_p_tags(row[6]) 


        print(f'{(i+1)/len(list_meetings)*100:.2f}%', end='\r')
    write_list_pargraphes(list_pt)

In [15]:
%%time
extract_from_csv_p_tags('Text/list_meetings.csv')

CPU times: user 54.1 s, sys: 2.34 s, total: 56.5 s
Wall time: 10min 30s
